In [1]:
import os
import torch
import torchvision
!pip install torchsummary
import torch.nn as nn
from torchvision import models
from torchsummary import summary

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
# minibatch size
batch_size = 4

# PreProcessing

In [4]:
from torchvision import transforms
from PIL import Image


# Define resize operation if needed
resize = transforms.Resize((48, 48))  # Replace with (96, 96) if upsizing is desired

# Define transformations for training, validation, and testing
trainTransforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
    resize,
    transforms.RandomHorizontalFlip(),  # Data augmentation
    transforms.RandomRotation(10),  # Data augmentation
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),  # Normalizing for grayscale image
])

validationTransforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
    resize,
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])

testTransforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
    resize,
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])

In [5]:
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.data import random_split




# Load the dataset from the image folder
dataset = datasets.ImageFolder(root='/kaggle/input/fer-2013/train', transform=trainTransforms)

# Calculate the sizes for train and validation sets
train_size = int(0.7 * len(dataset))
val_size = len(dataset) - train_size

# Split the dataset
train_data, validation_data = random_split(dataset, [train_size, val_size])
# Setup the batch size hyperparameter
BATCH_SIZE = batch_size
test_data = ImageFolder('/kaggle/input/fer-2013/test', transform=testTransforms)

# Turn datasets into iterables (batches)
train_dataloader = DataLoader(train_data,
                              batch_size=BATCH_SIZE,
                              shuffle=True)

validation_dataloader = DataLoader(validation_data,
                              batch_size=BATCH_SIZE,
                              shuffle=True)

test_dataloader = DataLoader(test_data,
                             batch_size=BATCH_SIZE,
                             shuffle=False)


# Let's check out what we've created
print(f"Dataloaders: {train_dataloader, test_dataloader}")
print(f"Length of train dataloader: {len(train_dataloader)} batches of {BATCH_SIZE}")
print(f"Length of Validation dataloader: {len(validation_dataloader)} batches of {BATCH_SIZE}")
print(f"Length of test dataloader: {len(test_dataloader)} batches of {BATCH_SIZE}")

Dataloaders: (<torch.utils.data.dataloader.DataLoader object at 0x7dd941c33910>, <torch.utils.data.dataloader.DataLoader object at 0x7dd941c33c70>)
Length of train dataloader: 5024 batches of 4
Length of Validation dataloader: 2154 batches of 4
Length of test dataloader: 1795 batches of 4


In [6]:
nb_train_samples = 5024 * 4
nb_validation_samples = 2154 * 4
nb_test_samples = 1795 * 4

In [15]:
import torch
import torch.nn as nn

# Defining the neural network
class FIVE_CNN(nn.Module):
    def __init__(self):
        super(FIVE_CNN, self).__init__()
        
        # Convolutional blocks: Conv layer => BatchNorm => ReLU
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.conv_block3 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.conv_block4 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.conv_block5 = nn.Sequential(
            nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3, padding=1),
            nn.BatchNorm2d(1024),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # Adaptive average pooling
        self.adaptive_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        
        # Fully connected layer
        self.fc = nn.Linear(1024, 7) 
        
  

    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = self.conv_block4(x)
        x = self.conv_block5(x)
        x = self.adaptive_avg_pool(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.fc(x)
        return x

# Instantiating the model
model = FIVE_CNN()



# FOCAL LOSS

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, gamma=2, alpha=None):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha

    def forward(self, input, target):
        ce_loss = F.cross_entropy(input, target, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = (1 - pt) ** self.gamma * ce_loss
        if self.alpha is not None:
            focal_loss = self.alpha * focal_loss
        return focal_loss.mean()

# Usage example:
# Instantiate Focal Loss
criterion = FocalLoss(gamma=2, alpha=None)

# Then, in your training loop, use this `criterion` as your loss function:
# loss = criterion(outputs, labels)


In [9]:
import torch
import torch.optim as optim
from torch.autograd import Variable
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Criterion and Optimizer
#criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, nesterov=True)
model=model.to(device)
# Scheduler
scheduler = ReduceLROnPlateau(optimizer, 'max', factor=0.75, patience=5, verbose=True)

num_epochs = 25
print(torch.cuda.is_available())

for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(train_dataloader.dataset)
    epoch_acc = running_corrects.double() / len(train_dataloader.dataset)
    print(f'Epoch {epoch}/{num_epochs - 1} - Training loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')

    # Validation phase
    model.eval()
    val_running_loss = 0.0
    val_running_corrects = 0
    with torch.no_grad():
        for inputs, labels in validation_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)
            val_running_loss += loss.item() * inputs.size(0)
            val_running_corrects += torch.sum(preds == labels.data)

    val_epoch_loss = val_running_loss / len(validation_dataloader.dataset)
    val_epoch_acc = val_running_corrects.double() / len(validation_dataloader.dataset)
    print(f'Epoch {epoch}/{num_epochs - 1} - Validation loss: {val_epoch_loss:.4f}, Accuracy: {val_epoch_acc:.4f}')

    # Adjust learning rate based on validation accuracy
    scheduler.step(val_epoch_acc)

# Save the model after training
torch.save(model.state_dict(), 'vggnet_fer.pth')


True
Epoch 0/24 - Training loss: 1.2278, Accuracy: 0.3029
Epoch 0/24 - Validation loss: 1.0638, Accuracy: 0.3831
Epoch 1/24 - Training loss: 1.0320, Accuracy: 0.3980
Epoch 1/24 - Validation loss: 0.9593, Accuracy: 0.4306
Epoch 2/24 - Training loss: 0.9550, Accuracy: 0.4368
Epoch 2/24 - Validation loss: 0.9086, Accuracy: 0.4602
Epoch 3/24 - Training loss: 0.8980, Accuracy: 0.4600
Epoch 3/24 - Validation loss: 0.8588, Accuracy: 0.4806
Epoch 4/24 - Training loss: 0.8623, Accuracy: 0.4800
Epoch 4/24 - Validation loss: 0.8442, Accuracy: 0.4737
Epoch 5/24 - Training loss: 0.8437, Accuracy: 0.4908
Epoch 5/24 - Validation loss: 0.8326, Accuracy: 0.5091
Epoch 6/24 - Training loss: 0.8149, Accuracy: 0.5004
Epoch 6/24 - Validation loss: 0.8090, Accuracy: 0.5093
Epoch 7/24 - Training loss: 0.7910, Accuracy: 0.5151
Epoch 7/24 - Validation loss: 0.8501, Accuracy: 0.4720
Epoch 8/24 - Training loss: 0.7799, Accuracy: 0.5200
Epoch 8/24 - Validation loss: 0.8379, Accuracy: 0.4860
Epoch 9/24 - Training l

In [10]:
model.eval()  # Set the model to evaluation mode
test_loss = 0.0
test_corrects = 0

# You should define nb_test_samples before this block
# It should be the total number of samples in the test set
nb_test_samples = len(test_dataloader.dataset)

with torch.no_grad():
    for inputs, labels in test_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * inputs.size(0)

        _, preds = torch.max(outputs, 1)
        test_corrects += torch.sum(preds == labels.data).item()

# Calculate the average loss and accuracy
test_epoch_loss = test_loss / nb_test_samples
test_epoch_accuracy = test_corrects / nb_test_samples
test_accuracy_percentage = test_epoch_accuracy * 100

print(f'Test loss: {test_epoch_loss:.4f}')
print(f'Test Accuracy: {test_accuracy_percentage:.2f}%')


Test loss: 0.6841
Test Accuracy: 56.91%


# Kullback Leibler Divergence Loss 

In [16]:
import torch
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Define Kullback-Leibler Divergence (KLD) Loss Function
import torch.nn.functional as F

def kld_loss(outputs, labels):
    log_probs = F.log_softmax(outputs, dim=1)
    target_probs = F.one_hot(labels, num_classes=outputs.size(1))  # Convert labels to one-hot vectors
    target_probs = target_probs.float()
    return F.kl_div(log_probs, target_probs, reduction='batchmean')
 

# Adam Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)


# Number of epochs
num_epochs = 25



# Move model to GPU if available
model = model.to(device)

# Training Loop
for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = kld_loss(outputs, labels)  # Use KLD loss here
        _, preds = torch.max(outputs, 1)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(train_dataloader.dataset)
    epoch_acc = running_corrects.double() / len(train_dataloader.dataset)
    print(f'Epoch {epoch}/{num_epochs - 1} - Training loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')

    # Validation phase
    model.eval()
    val_running_loss = 0.0
    val_running_corrects = 0
    
    with torch.no_grad():
        for inputs, labels in validation_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = kld_loss(outputs, labels)  # Use KLD loss here
            _, preds = torch.max(outputs, 1)
            val_running_loss += loss.item() * inputs.size(0)
            val_running_corrects += torch.sum(preds == labels.data)

    val_epoch_loss = val_running_loss / len(validation_dataloader.dataset)
    val_epoch_acc = val_running_corrects.double() / len(validation_dataloader.dataset)
    print(f'Epoch {epoch}/{num_epochs - 1} - Validation loss: {val_epoch_loss:.4f}, Accuracy: {val_epoch_acc:.4f}')

    

# Save the model after training
torch.save(model.state_dict(), 'vggnet_kld_adam.pth')


Epoch 0/24 - Training loss: 1.6253, Accuracy: 0.3607
Epoch 0/24 - Validation loss: 1.4222, Accuracy: 0.4502
Epoch 1/24 - Training loss: 1.3233, Accuracy: 0.4885
Epoch 1/24 - Validation loss: 1.2796, Accuracy: 0.5170
Epoch 2/24 - Training loss: 1.2153, Accuracy: 0.5355
Epoch 2/24 - Validation loss: 1.1940, Accuracy: 0.5435
Epoch 3/24 - Training loss: 1.1408, Accuracy: 0.5670
Epoch 3/24 - Validation loss: 1.1699, Accuracy: 0.5590
Epoch 4/24 - Training loss: 1.0834, Accuracy: 0.5911
Epoch 4/24 - Validation loss: 1.1400, Accuracy: 0.5594
Epoch 5/24 - Training loss: 1.0271, Accuracy: 0.6102
Epoch 5/24 - Validation loss: 1.1413, Accuracy: 0.5762
Epoch 6/24 - Training loss: 0.9770, Accuracy: 0.6307
Epoch 6/24 - Validation loss: 1.1066, Accuracy: 0.5776
Epoch 7/24 - Training loss: 0.9334, Accuracy: 0.6488
Epoch 7/24 - Validation loss: 1.1030, Accuracy: 0.5886
Epoch 8/24 - Training loss: 0.8760, Accuracy: 0.6690
Epoch 8/24 - Validation loss: 1.1300, Accuracy: 0.5825
Epoch 9/24 - Training loss: 

In [17]:
model.eval()  # Set the model to evaluation mode
test_loss = 0.0
test_corrects = 0

# You should define nb_test_samples before this block
# It should be the total number of samples in the test set
nb_test_samples = len(test_dataloader.dataset)

with torch.no_grad():
    for inputs, labels in test_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * inputs.size(0)

        _, preds = torch.max(outputs, 1)
        test_corrects += torch.sum(preds == labels.data).item()

# Calculate the average loss and accuracy
test_epoch_loss = test_loss / nb_test_samples
test_epoch_accuracy = test_corrects / nb_test_samples
test_accuracy_percentage = test_epoch_accuracy * 100

print(f'Test loss: {test_epoch_loss:.4f}')
print(f'Test Accuracy: {test_accuracy_percentage:.2f}%')


Test loss: 1.5340
Test Accuracy: 62.55%


#